# Visualisation de données en graphes : Neo4j

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows",250)
pd.set_option("display.max_columns",250)

from py2neo import Graph, authenticate, Relationship
authenticate("localhost:7474", "neo4j", "admin")

In [2]:
graph = Graph()

## Import des dataset

In [3]:
workers = pd.read_csv("attack_worker_demographics.tsv", delimiter="\t")
workers.head()

,worker_id,gender,english_first_language,age_group,education
0,833,female,0,45-60,bachelors
1,1072,male,0,30-45,bachelors
2,872,male,0,18-30,hs
3,2116,male,0,30-45,professional
4,453,male,0,30-45,hs


In [4]:
revisions = pd.read_csv("attack_annotated_comments.tsv", delimiter="\t")
revisions.head()

,rev_id,comment,year,logged_in,ns,sample,split
0,37675,`-NEWLINE_TOKENThis is not ``creative``. Thos...,2002,False,article,random,train
1,44816,`NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...,2002,False,article,random,train
2,49851,"NEWLINE_TOKENNEWLINE_TOKENTrue or false, the s...",2002,False,article,random,train
3,89320,"Next, maybe you could work on being less cond...",2002,True,article,random,dev
4,93890,This page will need disambiguation.,2002,True,article,random,train


In [5]:
attacks = pd.read_csv("attack_annotations.tsv", delimiter="\t")
attacks.head()

,rev_id,worker_id,quoting_attack,recipient_attack,third_party_attack,other_attack,attack
0,37675,1362,0.0,0.0,0.0,0.0,0.0
1,37675,2408,0.0,0.0,0.0,0.0,0.0
2,37675,1493,0.0,0.0,0.0,0.0,0.0
3,37675,1439,0.0,0.0,0.0,0.0,0.0
4,37675,170,0.0,0.0,0.0,0.0,0.0


In [6]:
print str(len(workers)) + " utilsateurs, " + str(len(revisions)) + " commentaires et " + str(len(attacks)) + " analyses."


2190 utilsateurs, 115864 commentaires et 1365217 analyses.


## Format d'entrée pour Neo4j

In [7]:
# Workers

typeWorker = "Worker"
i = 0
createW = "CREATE ("+typeWorker+str(workers["worker_id"][i])+":"+typeWorker
worker_id =" {worker_id: "+str(workers["worker_id"][i])
gender = ", gender: '"+str(workers["gender"][i])
english_first_language = "', english_first_language: "+str(workers["english_first_language"][i])
age_group = ", age_group: '"+str(workers["age_group"][i])
education = "', education: '"+str(workers["education"][i])
attributes = worker_id + gender + english_first_language + age_group + education 
createW += attributes + "'})\n"


# Revisions

typeRev = "Revision"
createR = ""
i=0
createR = "CREATE ("+typeRev+str(revisions["rev_id"][i])+":"+typeRev
rev_id =" {rev_id: "+str(revisions["rev_id"][i])
# comment = ", comment:"+str(revisions["comment"][i])
year = ", year:"+str(revisions["year"][i])
logged_in = ", logged_in:"+str(revisions["logged_in"][i])
ns = ", ns: '"+str(revisions["ns"][i])
sample = "', sample: '"+str(revisions["sample"][i])
split = "', split: '"+str(revisions["split"][i])
attributes = rev_id + year + logged_in + ns + sample + split
createR += attributes + "'})\n"


# Attacks




typeWorker = "Worker"
typeRev = "Revision"
createT = ""
i=0
isQuotingAttack = str(int(attacks["quoting_attack"][i]))
isRecipientAttack = str(int(attacks["recipient_attack"][i]))
isThirdPartyAttack = str(int(attacks["third_party_attack"][i]))
isOtherAttack = str(int(attacks["other_attack"][i]))



createA = "\n"+ "MATCH (w:"+typeWorker+"),(rev:"+typeRev+")"
createA += "\n"+"WHERE w.worker_id = "+str(attacks["worker_id"][i]) +" AND rev.rev_id = "+str(attacks["rev_id"][i])
createA += "\n"+"CREATE (w)-[:ATTACKS_IN {"
quoting_attack = "quoting_attack: " + isQuotingAttack
recipient_attack = ", recipient_attack: "+ isRecipientAttack
third_party_attack = ", third_party_attack: " + isThirdPartyAttack
other_attack = ", other_attack: " + isOtherAttack
rev = "}]->(rev)"
attributes = quoting_attack + recipient_attack + third_party_attack + other_attack
createA += attributes 
createA += rev + "\n"

    

In [8]:
print createW
print createR
print createA

CREATE (Worker833:Worker {worker_id: 833, gender: 'female', english_first_language: 0, age_group: '45-60', education: 'bachelors'})

CREATE (Revision37675:Revision {rev_id: 37675, year:2002, logged_in:False, ns: 'article', sample: 'random', split: 'train'})


MATCH (w:Worker),(rev:Revision)
WHERE w.worker_id = 1362 AND rev.rev_id = 37675
CREATE (w)-[:ATTACKS_IN {quoting_attack: 0, recipient_attack: 0, third_party_attack: 0, other_attack: 0}]->(rev)



## Injection dans la base Neo4j

In [9]:
typeWorker = "Worker"

for i in range(len(workers)):
#for i in range(100):    
    createW = "CREATE ("+typeWorker+str(workers["worker_id"][i])+":"+typeWorker
    worker_id =" {worker_id: "+str(workers["worker_id"][i])
    gender = ", gender: '"+str(workers["gender"][i])
    english_first_language = "', english_first_language: "+str(workers["english_first_language"][i])
    age_group = ", age_group: '"+str(workers["age_group"][i])
    education = "', education: '"+str(workers["education"][i])
    attributes = worker_id + gender + english_first_language + age_group + education 
    createW += attributes + "'})\n"
    graph.run(createW)

typeRev = "Revision"
createR = ""
# for i in range(len(revisions)):
for i in range(2000) :
    createR = "CREATE ("+typeRev+str(revisions["rev_id"][i])+":"+typeRev
    rev_id =" {rev_id: "+str(revisions["rev_id"][i])
    # comment = ", comment:"+str(revisions["comment"][i])
    year = ", year:"+str(revisions["year"][i])
    logged_in = ", logged_in:"+str(revisions["logged_in"][i])
    ns = ", ns: '"+str(revisions["ns"][i])
    sample = "', sample: '"+str(revisions["sample"][i])
    split = "', split: '"+str(revisions["split"][i])
    attributes = rev_id + year + logged_in + ns + sample + split
    createR += attributes + "'})\n"
    graph.run(createR)
    
typeWorker = "Worker"
typeRev = "Revision"
createT = ""
#for i in range(10) :
#for i in range(len(attacks)) : 
for i in range(10000) : 
        isQuotingAttack = str(int(attacks["quoting_attack"][i]))
        isRecipientAttack = str(int(attacks["recipient_attack"][i]))
        isThirdPartyAttack = str(int(attacks["third_party_attack"][i]))
        isOtherAttack = str(int(attacks["other_attack"][i]))

        createA = "\n"+ "MATCH (w:"+typeWorker+"),(rev:"+typeRev+")"
        createA += "\n"+"WHERE w.worker_id = "+str(attacks["worker_id"][i]) +" AND rev.rev_id = "+str(attacks["rev_id"][i])
        createA += "\n"+"CREATE (w)-[:ATTACKS_IN {"
        quoting_attack = "quoting_attack: " + isQuotingAttack
        recipient_attack = ", recipient_attack: "+ isRecipientAttack
        third_party_attack = ", third_party_attack: " + isThirdPartyAttack
        other_attack = ", other_attack: " + isOtherAttack
        rev = "}]->(rev)"
        attributes = quoting_attack + recipient_attack + third_party_attack + other_attack
        createA += attributes 
        createA += rev + "\n"
        graph.run(createA)

## Requéter la base Neo4j


In [11]:
# Nombre d'utilisateurs
graph.run('''
MATCH (w:Worker)
RETURN COUNT(w) as NB_WORKER
''').data()

[{u'NB_WORKER': 2190}]

In [12]:
# Les 3 premiers workers
graph.run('''
MATCH (w:Worker)
RETURN w 
ORDER BY w.worker_id
LIMIT 3
''').data()

[{u'w': (b336160:Worker {age_group:"18-30",education:"hs",english_first_language:0,gender:"male",worker_id:0})},
 {u'w': (de35e4a:Worker {age_group:"30-45",education:"hs",english_first_language:0,gender:"male",worker_id:4})},
 {u'w': (ec4f4e1:Worker {age_group:"18-30",education:"bachelors",english_first_language:0,gender:"male",worker_id:5})}]

In [13]:
# Nombre de commentaires
graph.run('''
MATCH (rev:Revision)
RETURN COUNT(rev) as NB_REVISION
''').data()

[{u'NB_REVISION': 2000}]

In [14]:
# Les 5 plus anciens commentaires
graph.run('''
MATCH (rev:Revision)
RETURN rev 
ORDER BY rev.year
LIMIT 5
''').data()

[{u'rev': (face807:Revision {logged_in:true,ns:"article",rev_id:249432,sample:"random",split:"train",year:2001})},
 {u'rev': (d23481f:Revision {logged_in:true,ns:"article",rev_id:268558,sample:"random",split:"train",year:2001})},
 {u'rev': (a11d460:Revision {logged_in:true,ns:"article",rev_id:252031,sample:"random",split:"train",year:2001})},
 {u'rev': (beb83e6:Revision {logged_in:true,ns:"article",rev_id:290598,sample:"random",split:"dev",year:2001})},
 {u'rev': (cb2fa2c:Revision {logged_in:true,ns:"article",rev_id:276906,sample:"random",split:"train",year:2001})}]

In [16]:
# Nombre d'attaques chargées dans la base Neo4j
graph.run('''
MATCH ()-[r:ATTACKS_IN]->() 
RETURN COUNT(r) as REL
''').data()

[{u'REL': 6414}]

In [17]:
# La liste des quoting_attacks
graph.run('''
MATCH (w)-[r:ATTACKS_IN]->() 
WHERE r.quoting_attack=1
RETURN r, w.worker_id
ORDER BY w.worker_id
LIMIT 10
''').data()

[{u'r': (a172dbb)-[:ATTACKS_IN {other_attack:0,quoting_attack:1,recipient_attack:0,third_party_attack:0}]->(e6aae8f),
  u'w.worker_id': 150},
 {u'r': (c2d296a)-[:ATTACKS_IN {other_attack:0,quoting_attack:1,recipient_attack:0,third_party_attack:0}]->(bf1e584),
  u'w.worker_id': 282},
 {u'r': (c17313d)-[:ATTACKS_IN {other_attack:0,quoting_attack:1,recipient_attack:0,third_party_attack:0}]->(ca91ebe),
  u'w.worker_id': 306},
 {u'r': (c17313d)-[:ATTACKS_IN {other_attack:0,quoting_attack:1,recipient_attack:0,third_party_attack:0}]->(aa05aaa),
  u'w.worker_id': 306},
 {u'r': (e10fd2d)-[:ATTACKS_IN {other_attack:0,quoting_attack:1,recipient_attack:0,third_party_attack:0}]->(e6aae8f),
  u'w.worker_id': 341},
 {u'r': (d7b6187)-[:ATTACKS_IN {other_attack:0,quoting_attack:1,recipient_attack:0,third_party_attack:0}]->(a8c15eb),
  u'w.worker_id': 476},
 {u'r': (e5e3bc1)-[:ATTACKS_IN {other_attack:0,quoting_attack:1,recipient_attack:0,third_party_attack:0}]->(eae07bc),
  u'w.worker_id': 490},
 {u'r'

In [18]:
# La liste des utilisateurs ayant réalisé une quoting_attack en 2002
graph.run('''
MATCH (w)-[r:ATTACKS_IN]->(rev) 
WHERE r.quoting_attack=1 AND rev.year=2002
RETURN w.worker_id
ORDER BY w.worker_id
LIMIT 10
''').data()

[{u'w.worker_id': 614}, {u'w.worker_id': 1622}]

In [19]:
# les années pour lesquelles le Worker 5 est associé à un commentaire
graph.run('''
MATCH (w {worker_id:5})-->(rev) 
RETURN  DISTINCT rev.year as YEAR_WITH_COMMENT
''').data()



[{u'YEAR_WITH_COMMENT': 2004},
 {u'YEAR_WITH_COMMENT': 2003},
 {u'YEAR_WITH_COMMENT': 2002}]

In [20]:
# Les utilisateurs associés à plusieurs attaques
graph.run('''
MATCH (w)-[r:ATTACKS_IN]->(rev: Revision) 
WITH COUNT(DISTINCT(rev.rev_id)) as NB_REV, w
RETURN w.worker_id, NB_REV 
ORDER BY NB_REV DESC
LIMIT 10
''').data()

[{u'NB_REV': 17, u'w.worker_id': 609},
 {u'NB_REV': 16, u'w.worker_id': 569},
 {u'NB_REV': 16, u'w.worker_id': 820},
 {u'NB_REV': 16, u'w.worker_id': 383},
 {u'NB_REV': 15, u'w.worker_id': 115},
 {u'NB_REV': 15, u'w.worker_id': 10},
 {u'NB_REV': 15, u'w.worker_id': 1936},
 {u'NB_REV': 15, u'w.worker_id': 11},
 {u'NB_REV': 15, u'w.worker_id': 69},
 {u'NB_REV': 15, u'w.worker_id': 970}]

In [21]:
# Quelle niveau scolaire des attaquants ?
graph.run('''
MATCH (w)-[r:ATTACKS_IN]->(rev: Revision) 
WHERE r.other_attack = 1 OR r.quoting_attack=1 OR r.recipient_attack=1 OR r.third_party_attack=1
RETURN w.worker_id as WORKER_ID, w.education as EDU
ORDER BY WORKER_ID 
LIMIT 10
''').data()

[{u'EDU': u'bachelors', u'WORKER_ID': 10},
 {u'EDU': u'professional', u'WORKER_ID': 12},
 {u'EDU': u'professional', u'WORKER_ID': 12},
 {u'EDU': u'professional', u'WORKER_ID': 12},
 {u'EDU': u'professional', u'WORKER_ID': 12},
 {u'EDU': u'masters', u'WORKER_ID': 24},
 {u'EDU': u'professional', u'WORKER_ID': 26},
 {u'EDU': u'bachelors', u'WORKER_ID': 30},
 {u'EDU': u'bachelors', u'WORKER_ID': 30},
 {u'EDU': u'professional', u'WORKER_ID': 44}]

In [22]:
# Et avec des stats
graph.run('''
MATCH (w)-[r:ATTACKS_IN]->(rev: Revision) 
WHERE r.other_attack = 1 OR r.quoting_attack=1 OR r.recipient_attack=1 OR r.third_party_attack=1
RETURN w.education AS EDU, count(w.education) AS NB_OCCURENCES
ORDER BY NB_OCCURENCES DESC
LIMIT 10
''').data()

[{u'EDU': u'bachelors', u'NB_OCCURENCES': 153},
 {u'EDU': u'hs', u'NB_OCCURENCES': 99},
 {u'EDU': u'masters', u'NB_OCCURENCES': 70},
 {u'EDU': u'professional', u'NB_OCCURENCES': 46},
 {u'EDU': u'some', u'NB_OCCURENCES': 13},
 {u'EDU': u'doctorate', u'NB_OCCURENCES': 1}]

In [35]:
graph.run('''
MATCH (w)-[r:ATTACKS_IN]->(rev: Revision) 
WHERE r.other_attack = 1 OR r.quoting_attack=1 OR r.recipient_attack=1 OR r.third_party_attack=1
RETURN w.education AS EDU, count(w.education) AS NB_EDU
ORDER BY NB_EDU DESC
LIMIT 1
''').data()

[{u'EDU': u'bachelors', u'NB_EDU': 153}]

In [34]:
graph.run('''
MATCH (w)-[r:ATTACKS_IN]->(rev: Revision) 
WHERE r.other_attack = 1 OR r.quoting_attack=1 OR r.recipient_attack=1 OR r.third_party_attack=1
RETURN w.gender AS GENDER, count(w.gender) AS NB_GENDER
ORDER BY NB_GENDER DESC
LIMIT 1
''').data()

[{u'GENDER': u'male', u'NB_GENDER': 225}]

In [36]:
graph.run('''
MATCH (w)-[r:ATTACKS_IN]->(rev: Revision) 
WHERE r.other_attack = 1 OR r.quoting_attack=1 OR r.recipient_attack=1 OR r.third_party_attack=1
RETURN w.english_first_language AS EFL, count(w.english_first_language) AS NB_EFL
ORDER BY NB_EFL DESC
LIMIT 1
''').data()

[{u'EFL': 0, u'NB_EFL': 309}]

In [37]:
graph.run('''
MATCH (w)-[r:ATTACKS_IN]->(rev: Revision) 
WHERE r.other_attack = 1 OR r.quoting_attack=1 OR r.recipient_attack=1 OR r.third_party_attack=1
RETURN w.age_group AS AGE, count(w.age_group) AS NB_AGE
ORDER BY NB_AGE DESC
LIMIT 1
''').data()

[{u'AGE': u'30-45', u'NB_AGE': 163}]

In [27]:
# Le profil type du méchant
graph.run('''
MATCH (w:Worker)-[r:ATTACKS_IN]->(rev: Revision)
WHERE w.gender = "male" AND w.english_first_language = 0 AND w.age_group = "30-45" AND w.education = "bachelors" 
AND r.other_attack = 1 OR r.quoting_attack=1 OR r.recipient_attack=1 OR r.third_party_attack=1
RETURN w.worker_id
''').data()

[{u'w.worker_id': 3341},
 {u'w.worker_id': 449},
 {u'w.worker_id': 2145},
 {u'w.worker_id': 3593},
 {u'w.worker_id': 979},
 {u'w.worker_id': 616},
 {u'w.worker_id': 1622},
 {u'w.worker_id': 306},
 {u'w.worker_id': 3784},
 {u'w.worker_id': 445},
 {u'w.worker_id': 46},
 {u'w.worker_id': 671},
 {u'w.worker_id': 1738},
 {u'w.worker_id': 1921},
 {u'w.worker_id': 227},
 {u'w.worker_id': 324},
 {u'w.worker_id': 2399},
 {u'w.worker_id': 1995},
 {u'w.worker_id': 605},
 {u'w.worker_id': 3931},
 {u'w.worker_id': 2918},
 {u'w.worker_id': 476},
 {u'w.worker_id': 1338},
 {u'w.worker_id': 1830},
 {u'w.worker_id': 1497},
 {u'w.worker_id': 614},
 {u'w.worker_id': 661},
 {u'w.worker_id': 1532},
 {u'w.worker_id': 189},
 {u'w.worker_id': 672},
 {u'w.worker_id': 58},
 {u'w.worker_id': 101},
 {u'w.worker_id': 1967},
 {u'w.worker_id': 2679},
 {u'w.worker_id': 3684},
 {u'w.worker_id': 2042},
 {u'w.worker_id': 3553},
 {u'w.worker_id': 893},
 {u'w.worker_id': 1298},
 {u'w.worker_id': 2408},
 {u'w.worker_id': 14